In [1]:
#default_exp model.msms

In [2]:
#hide
%reload_ext autoreload
%autoreload 2

In [3]:
#export
import torch
import pandas as pd
import numpy as np

from typing import List, Tuple

from tqdm import tqdm

from alphabase.peptide.fragment import \
    init_fragment_by_precursor_dataframe, \
    set_sliced_fragment_dataframe, \
    get_sliced_fragment_dataframe, \
    get_charged_frag_types

from alphadeep.model.featurize import \
    parse_aa_indices, parse_instrument_indices, \
    get_batch_mod_feature

from alphadeep._settings import \
    global_settings as settings, \
    model_const

import alphadeep.model.base as model_base

In [4]:
#export
class ModelMSMSpDeepTestResNet(torch.nn.Module):
    def __init__(self,
        num_ion_types,
        dropout=0.2,
        residue_conn=False,
    ):
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        BiRNN = True
        hidden=512
        hidden_rnn_layer=2

        self._residue_conn = residue_conn

        self.input_nn = model_base.InputAALSTM_cat_Meta(hidden)

        self.hidden_nn = model_base.SeqLSTM(
            hidden, hidden, rnn_layer=hidden_rnn_layer, 
            bidirectional=BiRNN
        )

        self.output_nn = model_base.OutputLSTM_cat_Meta(
            hidden,
            num_ion_types,
        )

    def forward(self,
        aa_indices,
        mod_x,
        charges:torch.Tensor,
        NCEs:torch.Tensor,
        instrument_indices,
    ):

        x = self.input_nn(
            aa_indices, mod_x, 
            charges, NCEs, instrument_indices
        )
        x = self.dropout(x)
        if self._residue_conn:
            res = x

        x = self.hidden_nn(x)
        x = self.dropout(x)

        if self._residue_conn:
            x *= res

        x = self.output_nn(x, charges, NCEs, instrument_indices)[:,3:,:]

        return x


In [5]:
#export
class ModelMSMSpDeep(torch.nn.Module):
    def __init__(self,
        num_ion_types,
        dropout=0.2,
    ):
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        BiRNN = True
        hidden=512
        hidden_rnn_layer=2

        self.input_nn = model_base.InputAALSTM_cat_Meta(hidden)

        self.hidden_nn = model_base.SeqLSTM(
            hidden, hidden, rnn_layer=hidden_rnn_layer, 
            bidirectional=BiRNN
        )

        self.output_nn = model_base.OutputLSTM_cat_Meta(
            hidden,
            num_ion_types,
        )

    def forward(self,
        aa_indices,
        mod_x,
        charges:torch.Tensor,
        NCEs:torch.Tensor,
        instrument_indices,
    ):

        x = self.input_nn(
            aa_indices, mod_x, 
            charges, NCEs, instrument_indices
        )
        x = self.dropout(x)

        x = self.hidden_nn(x)
        x = self.dropout(x)

        x = self.output_nn(x, charges, NCEs, instrument_indices)[:,3:,:]

        return x

In [6]:
#hide
import numpy as np
sequence = 'ACDEFGIK'
max_instrument_num = 4
num_ion_types=2
mod_feature_size=len(model_const['mod_elements'])
aa_indices = torch.LongTensor(parse_aa_indices([sequence]))
aa_indices = torch.nn.functional.pad(aa_indices, (1,1))
charges = torch.tensor([[2]])
instrument_indices = torch.LongTensor([1])
NCEs = torch.tensor([[0.3]])
mod_x = torch.zeros((1, aa_indices.size(1), mod_feature_size))

In [7]:
#hide
msms_model = ModelMSMSpDeep(num_ion_types)
msms_model

ModelMSMSpDeep(
  (dropout): Dropout(p=0.2, inplace=False)
  (input_nn): InputAALSTM_cat_Meta(
    (mod_nn): InputModNetFixFirstK(
      (nn): Linear(in_features=103, out_features=2, bias=False)
    )
    (meta_nn): InputMetaNet(
      (nn): Linear(in_features=9, out_features=3, bias=True)
    )
    (nn): SeqLSTM(
      (rnn): LSTM(35, 254, batch_first=True, bidirectional=True)
    )
  )
  (hidden_nn): SeqLSTM(
    (rnn): LSTM(512, 256, num_layers=2, batch_first=True, bidirectional=True)
  )
  (output_nn): OutputLSTM_cat_Meta(
    (meta_nn): InputMetaNet(
      (nn): Linear(in_features=9, out_features=3, bias=True)
    )
    (nn): SeqLSTM(
      (rnn): LSTM(516, 2, batch_first=True)
    )
  )
)

In [8]:
#hide
msms_model.eval()
msms_model(aa_indices, mod_x, charges, NCEs, instrument_indices)

tensor([[[-0.1990,  0.0918],
         [-0.1952,  0.0864],
         [-0.1931,  0.0836],
         [-0.1907,  0.0821],
         [-0.1891,  0.0818],
         [-0.1875,  0.0821],
         [-0.1869,  0.0817],
         [-0.1879,  0.0800],
         [-0.1921,  0.0749]]], grad_fn=<SliceBackward>)

# pDeepModel for MS/MS prediction

In [9]:
#export
class IntenAwareLoss(torch.nn.Module):
    def __init__(self, base_weight=0.2):
        super().__init__()
        self.w = base_weight

    def forward(self, pred, target):
        return torch.mean(
            (target+self.w)*torch.abs(target-pred)
        )

In [10]:
#export
mod_feature_size = len(model_const['mod_elements'])
max_instrument_num = model_const['max_instrument_num']
frag_types = settings['model']['frag_types']
max_frag_charge = settings['model']['max_frag_charge']
num_ion_types = len(frag_types)*max_frag_charge
nce_factor = model_const['nce_factor']
charge_factor = model_const['charge_factor']

class pDeepModel(model_base.ModelImplBase):
    def __init__(self,
        charged_frag_types = get_charged_frag_types(
            frag_types, max_frag_charge
        ),
        dropout=0.2,
        lr=0.001,
        model_class:torch.nn.Module=ModelMSMSpDeep,
        **kwargs, #model params
    ):
        super().__init__()
        self.charged_frag_types = charged_frag_types
        self.charge_factor = charge_factor
        self.NCE_factor = nce_factor
        self.build(
            model_class, 
            num_ion_types = len(self.charged_frag_types),
            dropout=dropout,
            lr=lr,
            **kwargs, # other model params
        )
        # self.loss_func = IntenAwareLoss()
        self.loss_func = torch.nn.L1Loss()
        self.min_inten = 1e-4

    def _prepare_train_data_df(self, 
        precursor_df:pd.DataFrame,
        fragment_inten_df:pd.DataFrame=None,
    ):
        self.frag_inten_df = fragment_inten_df[self.charged_frag_types]
        if np.all(precursor_df['NCE'].values > 1):
            precursor_df['NCE'] = precursor_df['NCE']*self.NCE_factor

    def _prepare_predict_data_df(self,
        precursor_df:pd.DataFrame,
        reference_frag_df:pd.DataFrame=None,
    ):
        
        self.predict_df = init_fragment_by_precursor_dataframe(
            precursor_df, self.charged_frag_types, reference_frag_df
        )

        if np.all(precursor_df['NCE'].values > 1):
            precursor_df['NCE'] = precursor_df['NCE']*self.NCE_factor

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame, 
        nAA, **kwargs,
    ) -> Tuple[torch.Tensor]:
        aa_indices = torch.LongTensor(
            parse_aa_indices(
                batch_df['sequence'].values.astype('U')
            )
        )

        mod_x_batch = get_batch_mod_feature(batch_df, nAA)
        mod_x = torch.Tensor(mod_x_batch)

        charges = torch.Tensor(
            batch_df['charge'].values
        ).unsqueeze(1)*self.charge_factor

        nces = torch.Tensor(batch_df['NCE'].values).unsqueeze(1)

        instrument_indices = torch.LongTensor(
            parse_instrument_indices(batch_df['instrument'])
        )
        return aa_indices, mod_x, charges, nces, instrument_indices

    def _get_targets_from_batch_df(self, 
        batch_df: pd.DataFrame, nAA,
        fragment_inten_df:pd.DataFrame=None
    ) -> torch.Tensor:
        return torch.Tensor(
            get_sliced_fragment_dataframe(
                fragment_inten_df, 
                batch_df[
                    ['frag_start_idx','frag_end_idx']
                ].values
            ).values
        ).view(-1, nAA-1, len(self.charged_frag_types))

    def _set_batch_predict_data(self, 
        batch_df: pd.DataFrame, 
        predicts:np.array,
        **kwargs,
    ):
        predicts = predicts.clip(max=1)
        predicts[predicts<self.min_inten] = 0
        set_sliced_fragment_dataframe(
            self.predict_df,
            predicts.reshape(
                (-1, len(self.charged_frag_types))
            ),
            batch_df[
                ['frag_start_idx','frag_end_idx']
            ].values,
            self.charged_frag_types
        )

In [11]:
#export
class pDeepParamSearch(pDeepModel):
    '''
    Only for model training and selection step
    '''
    def __init__(self,
        model_class:torch.nn.Module=ModelMSMSpDeep,
        **kwargs
    ):
        lr = kwargs['lr']
        dropout = kwargs['dropout']
        del kwargs['lr']
        del kwargs['dropout']
        super().__init__(
            lr=lr,
            model_class = model_class,
            **kwargs,
        )
        if 'L1' in kwargs:
            if kwargs['L1']:
                self.loss_func = torch.nn.L1Loss()
            else:
                self.loss_func = IntenAwareLoss()
            
        self.save_as = f'msms_{"_".join([str(k)+"_"+str(v) for k,v in kwargs.items()])}.pth'
        
import itertools
def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        _dict = dict(zip(keys, instance))
        yield _dict

def get_param_iter(dropout=[0.1], lr=[0.001], **candidate_param_kwargs):
    """
    Args:
        candidate_param_kwargs: dict of lists. 
          Example: {'L1': [True, False], ...}
    """
    return product_dict(
        dropout=dropout, lr=lr,
        **candidate_param_kwargs
    )

In [12]:
#export

def pearson(x, y):
    return torch.cosine_similarity(
        x-x.mean(dim=1, keepdim=True),
        y-y.mean(dim=1, keepdim=True),
        dim = 1
    )

def spectral_angle(cos):
    cos[cos>1] = 1
    return 1 - 2 * torch.arccos(cos) / np.pi

def _get_ranks(x: torch.Tensor, device) -> torch.Tensor:
    sorted_idx = x.argsort(dim=1)
    flat_idx = (
        sorted_idx+torch.arange(
            x.size(0), device=device
        ).unsqueeze(1)*x.size(1)
    ).flatten()
    ranks = torch.zeros_like(flat_idx)
    ranks[flat_idx] = torch.arange(
        x.size(1), device=device
    ).unsqueeze(0).repeat(x.size(0),1).flatten()
    ranks = ranks.reshape(x.size())
    ranks[x==0] = 0
    return ranks

def spearman(x: torch.Tensor, y: torch.Tensor, device):
    """Compute correlation between 2 a batch of 1-D tensors
    Args:
        x: Shape (Batch, x)
        y: Shape (Batch, X)
    """
    x_rank = _get_ranks(x, device)
    y_rank = _get_ranks(y, device)
    
    n = x.size(1)
    upper = 6 * torch.sum((x_rank - y_rank).pow(2), dim=1)
    down = n * (n ** 2 - 1.0)
    return 1.0 - (upper / down)

def evaluate_msms(
    psm_df: pd.DataFrame,
    predict_inten_df: pd.DataFrame,
    fragment_inten_df: pd.DataFrame,
    charged_frag_types: List=None,
    metrics = ['PCC','COS','SA','SPC'],
    GPU = True,
    batch_size=10240,
    verbose=False,
)->pd.DataFrame:

    if torch.cuda.is_available() and GPU:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    if not charged_frag_types:
        charged_frag_types = fragment_inten_df.columns.values

    _grouped = psm_df.groupby('nAA')

    if verbose:
        batch_tqdm = tqdm(_grouped)
    else:
        batch_tqdm = _grouped

    for met in metrics:
        psm_df[met] = 0

    for nAA, df_group in batch_tqdm:
        for i in range(0, len(df_group), batch_size):   
            batch_end = i+batch_size
            batch_df = df_group.iloc[i:batch_end,:]

            pred_intens = torch.Tensor(
                get_sliced_fragment_dataframe(
                    predict_inten_df, 
                    batch_df[
                        ['frag_start_idx','frag_end_idx']
                    ].values,
                    charged_frag_types
                ).values
            ).reshape(
                -1, (nAA-1)*len(charged_frag_types)
            ).to(device)

            frag_intens = torch.Tensor(
                get_sliced_fragment_dataframe(
                    fragment_inten_df, 
                    batch_df[
                        ['frag_start_idx','frag_end_idx']
                    ].values,
                    charged_frag_types
                ).values
            ).reshape(
                -1, (nAA-1)*len(charged_frag_types)
            ).to(device)

            if 'PCC' in metrics:
                psm_df.loc[batch_df.index,'PCC'] = pearson(
                    pred_intens, frag_intens
                ).cpu().detach().numpy()

            if 'COS' in metrics or 'SA' in metrics:
                cos = torch.cosine_similarity(
                    pred_intens, frag_intens, dim=1
                )
                psm_df.loc[
                    batch_df.index,'COS'
                ] = cos.cpu().detach().numpy()
                
                if 'SA' in metrics:
                    psm_df.loc[
                        batch_df.index,'SA'
                    ] = spectral_angle(
                        cos
                    ).cpu().detach().numpy()

            if 'SPC' in metrics:
                psm_df.loc[batch_df.index,'SPC'] = spearman(
                    pred_intens, frag_intens, device
                ).cpu().detach().numpy()

    metrics_describ = psm_df[metrics].describe()
    add_cutoff_metric(metrics_describ, psm_df, thres=0.9)
    add_cutoff_metric(metrics_describ, psm_df, thres=0.75)

    torch.cuda.empty_cache()
    return psm_df, metrics_describ

def add_cutoff_metric(
    metrics_describ, metrics_df, thres=0.9
):
    vals = []
    for col in metrics_describ.columns.values:
        vals.append(metrics_df.loc[metrics_df[col]>thres, col].count()/len(metrics_df))
    metrics_describ.loc[f'>{thres:.2f}'] = vals
    return metrics_describ


### Examples

In [13]:
#hide
fragment_inten_df = pd.DataFrame({'b':np.arange(100)/100, 'y':np.arange(100)/100})
repeat = 10
precursor_df = pd.DataFrame({
    'sequence': ['AGHCEWQMKYR']*repeat,
    'mods': ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*repeat,
    'mod_sites': ['0;4;8']*repeat,
    'nAA': [11]*repeat,
    'NCE': [20]*repeat,
    'instrument': 'QE',
    'charge': np.arange(1,repeat+1),
    'frag_start_idx':np.arange(10, dtype=int)*10,
    'frag_end_idx':np.arange(10, dtype=int)*10+10,
})
precursor_df

,sequence,mods,mod_sites,nAA,NCE,instrument,charge,frag_start_idx,frag_end_idx
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,1,0,10
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,2,10,20
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,3,20,30
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,4,30,40
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,5,40,50
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,6,50,60
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,7,60,70
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,8,70,80
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,9,80,90
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,90,100


In [14]:
#hide
fragment_inten_df

,b,y
0,0.00,0.00
1,0.01,0.01
2,0.02,0.02
3,0.03,0.03
4,0.04,0.04
...,...,...
95,0.95,0.95
96,0.96,0.96
97,0.97,0.97
98,0.98,0.98


### Train an MSMS model

In [15]:
#hide
pdeep = pDeepModel()
pdeep.charged_frag_types=['b','y']
pdeep.build(
    ModelMSMSpDeep,
    num_ion_types = len(pdeep.charged_frag_types),
    dropout=0.2,
    lr=1e-3
)
pdeep.train(precursor_df, epoch=5, batch_size=3, fragment_inten_df=fragment_inten_df)

Epoch=5, nAA=11, Batch=4, Loss=0.1794: 100%|██████████| 1/1 [00:00<00:00,  4.04it/s]


### Predict from the model

In [16]:
#hide
predict_inten_df = pdeep.predict(precursor_df,reference_frag_df=fragment_inten_df)
predict_inten_df

,b,y
0,0.356518,0.301886
1,0.346430,0.276368
2,0.338893,0.258964
3,0.337007,0.251681
4,0.338530,0.252706
...,...,...
95,0.540719,0.872861
96,0.538788,0.866614
97,0.534705,0.849702
98,0.526209,0.816325


In [17]:
#hide
predict_inten_df = pdeep.predict(precursor_df,reference_frag_df=fragment_inten_df)
predict_inten_df

,b,y
0,0.356518,0.301886
1,0.346430,0.276368
2,0.338893,0.258964
3,0.337007,0.251681
4,0.338530,0.252706
...,...,...
95,0.540719,0.872861
96,0.538788,0.866614
97,0.534705,0.849702
98,0.526209,0.816325


### Metrics

In [18]:
#hide
_frag_inten_df = fragment_inten_df.copy()
_frag_inten_df.loc[0:0,:] = 0.1

In [19]:
#hide
df, metrics_describ = evaluate_msms(precursor_df, fragment_inten_df, _frag_inten_df, ['b','y'])
precursor_df

,sequence,mods,mod_sites,nAA,NCE,instrument,charge,frag_start_idx,frag_end_idx,PCC,COS,SA,SPC
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,1,0,10,0.454545,0.860383,0.659552,0.424812
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,2,10,20,1.000000,1.000000,1.000000,1.000000
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,3,20,30,1.000000,1.000000,1.000000,1.000000
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,4,30,40,1.000000,1.000000,1.000000,1.000000
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,5,40,50,1.000000,1.000000,1.000000,1.000000
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,6,50,60,1.000000,1.000000,1.000000,1.000000
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,7,60,70,1.000000,1.000000,1.000000,1.000000
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,8,70,80,1.000000,1.000000,1.000000,1.000000
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,9,80,90,1.000000,1.000000,1.000000,1.000000
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.2,QE,10,90,100,1.000000,1.000000,1.000000,1.000000


In [20]:
#hide
metrics_describ

,PCC,COS,SA,SPC
count,10.000000,10.000000,10.000000,10.000000
mean,0.945455,0.986038,0.965955,0.942481
std,0.172488,0.044151,0.107659,0.181890
min,0.454545,0.860383,0.659552,0.424812
25%,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000
>0.90,0.900000,0.900000,0.900000,0.900000
>0.75,0.900000,1.000000,0.900000,0.900000
